In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [3]:
# Example data
c_x = np.array([3, 2])         # Objective coefficients for x
c_y = np.array([4, 1])         # Objective coefficients for y

A_x = np.array([[2, 1],        # Coefficient matrix for x in constraints
                [1, 2]])
A_y = np.array([[1, 2],        # Coefficient matrix for y in constraints
                [2, 1]])

b = np.array([20, 30])         # Right-hand side for constraints
k = 10                         # Bilinear constraint bound (x_i * y_j <= k)

# Sizes of the x and y vectors
n_x = len(c_x)
n_y = len(c_y)

# Create a new model
model = gp.Model("bilinear_problem")

# Define variables for x and y (x >= 0, y >= 0)
x = model.addMVar(shape=n_x, name="x", lb=0)
y = model.addMVar(shape=n_y, name="y", lb=0)

# Set the objective function (maximize c_x^T x + c_y^T y)
model.setObjective(c_x @ x + c_y @ y, GRB.MAXIMIZE)

# Add constraints (A_x x + A_y y <= b)
model.addMConstr(A_x, x, "<=", b)
model.addMConstr(A_y, y, "<=", b)

# Add auxiliary variables for the bilinear terms
bilinear_vars = []
for i in range(n_x):
    for j in range(n_y):
        bilinear_var = model.addVar(name=f"bilinear_{i}_{j}", lb=0)
        bilinear_vars.append(bilinear_var)
        
        # Add bilinear constraints: bilinear_var = x[i] * y[j]
        model.addConstr(bilinear_var == x[i] * y[j], name=f"bilinear_eq_{i}_{j}")
        
        # Add upper bound for the bilinear term: bilinear_var <= k
        model.addConstr(bilinear_var <= k, name=f"bilinear_bound_{i}_{j}")

# Optimize the model
model.optimize()

# Print the solution
if model.status == GRB.OPTIMAL:
    print(f"Optimal solution for x: {x.X}")
    print(f"Optimal solution for y: {y.X}")
    print(f"Optimal objective value: {model.objVal}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-22


GurobiError: Invalid argument to Model.addConstr